## Preprocessing

In [74]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [75]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [76]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [77]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [78]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
cutoff_value = 500
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [79]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [80]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter the counts to only include those greater than 1
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [81]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [82]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,True,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
4,1,142590,1,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False


In [83]:
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)


In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [85]:
# Importing necessary Keras modules
from tensorflow.keras.layers import Dense

# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer with 80 nodes and ReLU activation
nn.add(Dense(units=80, input_dim=input_features, activation='relu'))

# Second hidden layer with 30 nodes and ReLU activation
nn.add(Dense(units=30, activation='relu'))

# Output layer with 1 node and sigmoid activation
nn.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 80)                2960      
                                                                 
 dense_15 (Dense)            (None, 30)                2430      
                                                                 
 dense_16 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5421 (21.18 KB)
Trainable params: 5421 (21.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [87]:
# Train the model
# Step 8: Create a callback that saves the model's weights every five epochs.
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Step 9: Train the model (we're re-training for the sake of this demonstration)
fit_model = nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=100,
    callbacks=[cp_callback]
)


Epoch 1/100
804/804 [==============================] - 1s 730us/step - loss: 0.5742 - accuracy: 0.7182
Epoch 2/100
804/804 [==============================] - 1s 710us/step - loss: 0.5571 - accuracy: 0.7268
Epoch 3/100
804/804 [==============================] - 1s 722us/step - loss: 0.5545 - accuracy: 0.7291
Epoch 4/100
804/804 [==============================] - 1s 727us/step - loss: 0.5523 - accuracy: 0.7299
Epoch 5/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5508 - accuracy: 0.7289
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 731us/step - loss: 0.5504 - accuracy: 0.7294
Epoch 6/100
804/804 [==============================] - 1s 711us/step - loss: 0.5502 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 740us/step - loss: 0.5498 - accuracy: 0.7305
Epoch 8/100
804/804 [==============================] - 1s 707us/step - loss: 0.5484 - accuracy: 0.7307
Epoch 9/100
804/804 [=================

In [88]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5599 - accuracy: 0.7298 - 202ms/epoch - 753us/step
Loss: 0.5599150061607361, Accuracy: 0.7297959327697754


In [89]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

c:\Users\perry\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model Version 1.1 
## Increased epoch count, extra neurons/layer and and additional hidden layer

In [90]:
# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn_1 = tf.keras.models.Sequential(name="nn_V1.1")

# First hidden layer with 100 nodes and ReLU activation
nn_1.add(Dense(units=100, input_dim=input_features, activation='relu'))

# Second hidden layer with 50 nodes and ReLU activation
nn_1.add(Dense(units=50, activation='relu'))

# Third hidden layer with 50 nodes and ReLU activation
nn_1.add(Dense(units=50, activation='sigmoid'))

# Output layer with 1 node and sigmoid activation
nn_1.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn_1.summary()


Model: "nn_V1.1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 100)               3700      
                                                                 
 dense_18 (Dense)            (None, 50)                5050      
                                                                 
 dense_19 (Dense)            (None, 50)                2550      
                                                                 
 dense_20 (Dense)            (None, 1)                 51        
                                                                 
Total params: 11351 (44.34 KB)
Trainable params: 11351 (44.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [91]:
# Compile the model
nn_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [92]:
# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.1/", exist_ok=True)
checkpoint_path = "checkpointsV1.1/weights.{epoch:02d}.hdf5"

# Step 9: Train the model (we're re-training for the sake of this demonstration)
fit_model = nn_1.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)


Epoch 1/150


804/804 [==============================] - 1s 773us/step - loss: 0.5723 - accuracy: 0.7184
Epoch 2/150
804/804 [==============================] - 1s 754us/step - loss: 0.5552 - accuracy: 0.7287
Epoch 3/150
804/804 [==============================] - 1s 840us/step - loss: 0.5529 - accuracy: 0.7305
Epoch 4/150
804/804 [==============================] - 1s 766us/step - loss: 0.5504 - accuracy: 0.7305
Epoch 5/150
799/804 [============================>.] - ETA: 0s - loss: 0.5494 - accuracy: 0.7308
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 787us/step - loss: 0.5493 - accuracy: 0.7310
Epoch 6/150
804/804 [==============================] - 1s 790us/step - loss: 0.5488 - accuracy: 0.7306
Epoch 7/150
804/804 [==============================] - 1s 743us/step - loss: 0.5480 - accuracy: 0.7320
Epoch 8/150
804/804 [==============================] - 1s 710us/step - loss: 0.5477 - accuracy: 0.7320
Epoch 9/150
804/804 [=============================

In [93]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5807 - accuracy: 0.7290 - 210ms/epoch - 782us/step
Loss: 0.5806710124015808, Accuracy: 0.7289795875549316


In [94]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.1.h5")

# Model Version 1.2
##

In [95]:
from tensorflow.keras.layers import LeakyReLU

# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn_2 = tf.keras.models.Sequential(name="nn_V1.2")

# First hidden layer with 100 nodes and ReLU activation
nn_2.add(Dense(units=150, input_dim=input_features, activation='relu'))

# Second hidden layer with 50 nodes and ReLU activation
nn_2.add(Dense(units=100, activation='relu'))

# Third hidden layer with 50 nodes and ReLU activation
nn_2.add(Dense(units=128))
nn_2.add(LeakyReLU(alpha=0.01))  # alpha is the negative slope coefficient

# Output layer with 1 node and sigmoid activation
nn_2.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn_2.summary()

# Compile the model
nn_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.2/", exist_ok=True)
checkpoint_path = "checkpointsV1.2/weights.{epoch:02d}.hdf5"

# Step 9: Train the model
fit_model = nn_2.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.2.h5")

Model: "nn_V1.2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 150)               5550      
                                                                 
 dense_22 (Dense)            (None, 100)               15100     
                                                                 
 dense_23 (Dense)            (None, 128)               12928     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128)               0         
                                                                 
 dense_24 (Dense)            (None, 1)                 129       
                                                                 
Total params: 33707 (131.67 KB)
Trainable params: 33707 (131.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/150
804/804 [

# Model Version 1.3

In [67]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_3 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_3

In [68]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [73]:
# Run the kerastuner search for best hyperparameters without callbacks
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))


INFO:tensorflow:Oracle triggered exit


In [70]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 5,
 'num_layers': 5,
 'units_0': 3,
 'units_1': 5,
 'units_2': 5,
 'units_3': 3,
 'units_4': 7,
 'units_5': 5,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0052'}

In [71]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7326 - 295ms/epoch - 1ms/step
Loss: 0.5559583902359009, Accuracy: 0.7325947284698486


In [72]:
def build_best_model(hp_values):
    nn_best = tf.keras.models.Sequential()

    # Use the best activation function from hyperparameters
    activation = hp_values['activation']
    
    # Use the best number of neurons for the first layer
    nn_best.add(tf.keras.layers.Dense(units=hp_values['first_units'], activation=activation, input_dim=input_features))

    # Use the best number of hidden layers and neurons for each hidden layer
    for i in range(hp_values['num_layers']):
        nn_best.add(tf.keras.layers.Dense(units=hp_values['units_' + str(i)], activation=activation))
    
    nn_best.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_best.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_best

# Use the best hyperparameters to construct the model
best_model = build_best_model(best_hyper.values)
# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.3/", exist_ok=True)
checkpoint_path = "checkpointsV1.3/weights.{epoch:02d}.hdf5"

# Step 9: Train the model
fit_model = best_model.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)

# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.3.h5")

Epoch 1/150
804/804 [==============================] - 1s 801us/step - loss: 0.6058 - accuracy: 0.6964
Epoch 2/150
804/804 [==============================] - 1s 735us/step - loss: 0.5742 - accuracy: 0.7276
Epoch 3/150
804/804 [==============================] - 1s 746us/step - loss: 0.5668 - accuracy: 0.7286
Epoch 4/150
804/804 [==============================] - 1s 742us/step - loss: 0.5619 - accuracy: 0.7283
Epoch 5/150
761/804 [===========================>..] - ETA: 0s - loss: 0.5592 - accuracy: 0.7277
Epoch 5: saving model to checkpointsV1.3\weights.05.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5595 - accuracy: 0.7273
Epoch 6/150
804/804 [==============================] - 1s 735us/step - loss: 0.5577 - accuracy: 0.7296
Epoch 7/150
804/804 [==============================] - 1s 766us/step - loss: 0.5561 - accuracy: 0.7284
Epoch 8/150
804/804 [==============================] - 1s 758us/step - loss: 0.5555 - accuracy: 0.7294
Epoch 9/150
804/804 [=============